In [1]:
import requests
from bs4 import BeautifulSoup

<h2>Question 1</h2>

In [2]:
def extract_text_from_wikipedia(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.find_all('p')
    text = ' '.join([paragraph.get_text() for paragraph in paragraphs])
    return text

wikipedia_url = "https://en.wikipedia.org/wiki/Great_Hurricane_of_1780"

sentence = extract_text_from_wikipedia(wikipedia_url)

In [3]:
# sentence = 'Life is short, eat dessert first'

In [4]:
dc = {s:i for i,s in enumerate(sorted(sentence.replace(',', '').split()))}
print(dc)

{'"so': 0, '&': 1, "'Rock": 2, "'boiling": 3, "'divine": 4, "'injured'": 5, '("San': 6, '(150': 7, '(160': 8, '(295': 9, '(30': 10, '(320': 13, '(56': 14, '(7.6': 15, '(90': 16, '(San': 17, '(Santa': 18, '(except': 19, '(greater': 20, '(now': 21, '(the': 22, '...': 24, '1-minute': 25, '10': 29, '10.': 30, '100': 31, '10th]': 32, '11': 33, '11.[3]': 34, '114': 35, '11th;': 36, '12': 39, '12th': 40, '14': 42, '145': 43, '1492': 44, '15': 45, '16.[4]': 46, '1780': 47, '1780[2][1][3]': 48, '1851.[5]': 49, '1867': 50, '1899': 51, '18[3]': 52, '19': 53, '1928': 54, '1932': 55, '1953': 56, '1956)': 57, '1960': 58, '1960).[1]': 59, '1978': 60, '20': 61, '200': 64, '22': 65, '22000': 67, '240': 68, '25-foot': 69, '260': 70, '28500': 71, '40': 72, '4500': 73, '475': 74, '4:00': 75, '5': 77, '50': 78, '58': 79, '584': 80, '600': 81, '6000': 82, '6500': 83, '6:00': 84, '8:00': 85, '9.': 87, '90': 88, '9000': 90, 'A': 94, 'About': 95, 'According': 96, 'Admiral': 99, 'After': 100, 'Al': 101, 'Almigh

In [5]:
import torch

sentence_int = torch.tensor([dc[s] for s in sentence.replace(',', '').split()])
print(sentence_int)

tensor([442, 216, 252,  ..., 393,  77,  59])


In [6]:
torch.manual_seed(123)
embed = torch.nn.Embedding(len(sentence_int), 16)
embedded_sentence = embed(sentence_int).detach()

print(embedded_sentence)
print(embedded_sentence.shape)

tensor([[-0.8178,  0.8498, -0.1779,  ..., -0.6462,  1.0392, -0.6085],
        [ 0.0025, -0.1198, -0.3939,  ...,  1.3373,  0.2760,  1.1924],
        [ 0.3904,  0.0150, -0.6920,  ..., -1.3244,  1.3231,  1.4228],
        ...,
        [ 0.0434, -1.2346,  0.5360,  ...,  1.1410,  0.1475,  1.4444],
        [ 0.5383,  0.5548,  0.1676,  ...,  1.9944, -0.1130,  1.5312],
        [-0.5518,  0.8661,  0.1700,  ..., -0.6735, -0.5763, -0.9291]])
torch.Size([1725, 16])


In [7]:
torch.manual_seed(123)

d = embedded_sentence.shape[1]

d_q, d_k, d_v = 24, 24, 28

W_query = torch.nn.Parameter(torch.rand(d_q, d))
W_key = torch.nn.Parameter(torch.rand(d_k, d))
W_value = torch.nn.Parameter(torch.rand(d_v, d))

In [8]:
x_2 = embedded_sentence[1]
query_2 = W_query.matmul(x_2)
key_2 = W_key.matmul(x_2)
value_2 = W_value.matmul(x_2)

print(query_2.shape)
print(key_2.shape)
print(value_2.shape)

torch.Size([24])
torch.Size([24])
torch.Size([28])


In [9]:
keys = W_key.matmul(embedded_sentence.T).T
values = W_value.matmul(embedded_sentence.T).T

print("keys.shape:", keys.shape)
print("keys:", keys)

print("values.shape:", values.shape)
print("values:", values)

keys.shape: torch.Size([1725, 24])
keys: tensor([[-1.0686, -3.1086, -0.8889,  ..., -0.0201, -0.9175,  0.4097],
        [-0.5862,  0.1007, -0.3218,  ..., -0.4393,  1.3399,  0.8820],
        [-0.6226, -0.2295,  0.7695,  ...,  2.0305, -0.8997,  1.5053],
        ...,
        [-0.8296,  0.5317, -1.7287,  ..., -1.0672,  1.2302, -1.1580],
        [ 1.7435,  3.0576,  2.6530,  ...,  1.6043,  3.1653,  2.7961],
        [ 1.5013,  3.5477,  0.2387,  ...,  0.6648,  1.3506,  1.4122]],
       grad_fn=<PermuteBackward0>)
values.shape: torch.Size([1725, 28])
values: tensor([[-0.4749, -0.5429, -0.6069,  ..., -0.4019,  1.1915,  0.2306],
        [ 2.8718,  0.7893,  0.9213,  ...,  0.8229,  2.8054,  0.1350],
        [ 1.9336, -0.0849, -2.0625,  ..., -0.0727, -1.2707, -2.0734],
        ...,
        [ 0.9851,  1.9797,  1.0178,  ..., -0.4025, -1.6145, -2.4468],
        [ 4.5607,  3.6346,  3.0015,  ...,  2.7838,  3.3667,  1.8448],
        [ 1.2109, -0.5297, -0.0923,  ...,  1.4707,  2.0475,  2.0686]],
       grad

In [10]:
omega_24 = query_2.dot(keys[4])
print(omega_24)

tensor(-0.7693, grad_fn=<DotBackward0>)


In [11]:
omega_2 = query_2.matmul(keys.T)
print(omega_2)

tensor([-23.6476,  15.1263,  16.2770,  ...,   7.3063,  56.2660,   5.0327],
       grad_fn=<SqueezeBackward4>)


In [12]:
import torch.nn.functional as F

attention_weights_2 = F.softmax(omega_2 / d_k**0.5, dim=0)
print(attention_weights_2)

tensor([3.1426e-12, 8.6019e-09, 1.0879e-08,  ..., 1.7432e-09, 3.8162e-05,
        1.0960e-09], grad_fn=<SoftmaxBackward0>)


In [13]:
context_vector_2 = attention_weights_2.matmul(values)

print(context_vector_2.shape)
print(context_vector_2)

torch.Size([28])
tensor([6.4877, 6.5384, 5.5197, 4.8639, 4.4191, 9.7355, 3.7277, 4.6734, 4.7695,
        4.4612, 6.1600, 8.0053, 7.0177, 7.0056, 7.5777, 7.1089, 6.7831, 7.0839,
        4.7333, 6.4199, 6.5752, 7.4507, 6.6498, 5.4528, 5.7592, 5.6813, 5.4551,
        5.0441], grad_fn=<SqueezeBackward4>)


In [14]:
h = 3
multihead_W_query = torch.nn.Parameter(torch.rand(h, d_q, d))
multihead_W_key = torch.nn.Parameter(torch.rand(h, d_k, d))
multihead_W_value = torch.nn.Parameter(torch.rand(h, d_v, d))

In [15]:
multihead_query_2 = multihead_W_query.matmul(x_2)
print(multihead_query_2.shape)

torch.Size([3, 24])


In [16]:
multihead_key_2 = multihead_W_key.matmul(x_2)
multihead_value_2 = multihead_W_value.matmul(x_2)

In [17]:
stacked_inputs = embedded_sentence.T.repeat(3, 1, 1)
print(stacked_inputs.shape)

torch.Size([3, 16, 1725])


In [18]:
multihead_keys = torch.bmm(multihead_W_key, stacked_inputs)
multihead_values = torch.bmm(multihead_W_value, stacked_inputs)
print("multihead_keys.shape:", multihead_keys.shape)
print("multihead_values.shape:", multihead_values.shape)

multihead_keys.shape: torch.Size([3, 24, 1725])
multihead_values.shape: torch.Size([3, 28, 1725])


In [19]:
multihead_keys = multihead_keys.permute(0, 2, 1)
multihead_values = multihead_values.permute(0, 2, 1)
print("multihead_keys.shape:", multihead_keys.shape)
print("multihead_values.shape:", multihead_values.shape)

multihead_keys.shape: torch.Size([3, 1725, 24])
multihead_values.shape: torch.Size([3, 1725, 28])


<h2>Question 2</h2>

Tools used:
1) **SERP AI**: Wrapper around SerpAPI - a real-time API to access Google search results
2) **Calculator**: Perform calculations on response
3) **Buffer Memory**: Remembers previous conversational back and forths directly
4) **Chat OpenAI**: Wrapper around OpenAI large language models that use the Chat endpoint
5) **OpenAI Function Agent**: An agent that uses Function Calling to pick the tool and args to call



SERP AI and Calculator were fed into the `Allowed Tools` input of the OpenAI Function Agent, which allows it to use a simple calculator for  simple math calculations or use SERP AI for complex searches. There aren't any specific attributes that can be customized in this configuration apart from the LLM used and the function agent.